# Documentation of Equations and Workflow for Image Processing

## Workflow

1. Determine global histogram for raw images, allow user to see and choose limits
2. Read image, convert to float64
3. Determine core axis
4. Generate thickness model
5. Calculate murhot0
6. Stretch the histogram around the peak, invert and rescale to uint16

## Core Axis Determination

### Binary Image:
$
\text{The raw image is converted to a binary image}\\
I_{bin} = 
\begin{cases}
  1.0 & \quad \text{if } I_{raw} \le I_{threshold}\\
  0.0 & \quad \text{if } I_{raw} \gt I_{threshold}\\
\end{cases}
$

### Edge Detection:
$
\text{The binary image is converted to a lateral edge image}\\
I_{edge}\left(i,j\right) = I_{bin}\left(i,j+1\right) - I_{bin}\left(i,j\right)\\
\text{ }\\
\text{The result can be interpreted as follows}\\
I_{edge} = 
\begin{cases}
  1.0 & \quad \text{at the left edge of a feature}\\
  -1.0 & \quad \text{at the right edge of a feature}\\
\end{cases}
$

### Edges of Core Section:
$
\text{The core is assumed to be represented by the largest gap between values in the row}\\
$

### Midline of Core Section:
$
\text{The midpoint between the left and right edge in each row is determined}\\
\text{A linear regression of midline j's against i's is performed to determine the section axis}\\
$

## Thickness Modelling

### Assumptions:
The primary assumptions are that the cylinder axis is always at the same z and is aligned with the y axis. If it's not aligned with y, a simple xy axis rotation can be performed so that y' is aligned with the axis. This will keep the math the same (and simple). 

### Coordinate Systems:
$
\text{Convert from initial coordinates to rotated coordinates, where y' is aligned with cylinder axis}\\
x' = x\cos\theta - y\sin\theta \\
y' = x\sin\theta + y\cos\theta \\
z' = z
$

### Fundamental Equations:
$
\text{Equation of a cylinder, aligned with y' axis}\\
\left(x' - x'_{a}\right)^2 + \left(z' - z'_{a}\right)^2 = r^2\\
\text{ }\\
\text{Parametric equations of a ray}\\
x' = x'_{p} + \vec{u_{x'}}t\\
y' = y'_{p} + \vec{u_{y'}}t\\
z' = z'_{p} + \vec{u_{z'}}t
$

### Intersections Between a Ray and Horizontal Plane:
$
t_{Z} =
\begin{cases}
  \frac{(z'_{Z} - z'_{p})}{\vec{u_{z'}}} & \quad \text{if } \vec{u_{z'}} \ne 0\\
  \text{undefined} & \quad \text{if } \vec{u_{z'}} = 0\\
\end{cases} 
$

### Intersections Between a Ray and Cylinder:
$
\text{Substitution}\\
\left(x'_{p} + \vec{u_{x'}}t - x'_{a}\right)^2 + \left(z'_{p} + \vec{u_{z'}}t - z'_{a}\right)^2 = r^2\\
\text{ }\\
\text{Expand the Full Equation}\\
At^2 + Bt + C = 0\\
A = \vec{u_{x'}}^2 + \vec{u_{z'}}^2\\
B = 2\vec{u_{x'}}\left(x'_{p} - x'_{a}\right) + 2\vec{u_{z'}}\left(z'_{p} - z'_{a}\right)\\
C = {x'_{p}}^2 - 2x'_{p}x'_{a} + {x'_{a}}^2 + {z'_{p}}^2 - 2z'_{p}z'_{a} + {z'_{a}}^2 - r^2\\
\text{ }\\
\text{Solve for 't' Using the Quadratic Formula}\\
det = B^2 - 4AC\\
t_{c1}, t_{c2} =
\begin{cases}
  \frac{-B \pm \sqrt{det}}{2A} & \quad \text{if } det \ge 0\\
  \text{undefined} & \quad \text{if } det = 0\\
\end{cases}
$

### Path Lengths Through Whole Round and Half Round:
$
\text{Whole Round}\\
thickness =
\begin{cases}
  t_{c2} - t_{c1} & \quad \text{if } det \gt 0\\
  0 & \quad \text{if } det \le 0\\
\end{cases}
\text{ }\\
\text{Half Round}\\
thickness =
\begin{cases}
  t_{c2} - t_{c1} & \quad \text{if } det \gt 0 \text{ and } t_{Z} \lt t_{c1}\\
  t_{c2} - t_{Z} & \quad \text{if } det \gt 0 \text{ and } t_{c2} \gt t_{Z} \gt t_{c1}\\
  0 & \quad \text{if } det \le 0 \text{ or } t_{Z} \gt t_{c2}\\
\end{cases}
$

## Primary Calculations

### Raw Image:
$
I_{raw} \approx I_{0}\text{ }e^{-\mu\rho t}\\
\text{ }\\
\mu\rho = \text{effective value based on source spectrum and sample composition}\\
$

### Calculate murhot :
$
\mu\rho t = 
\begin{cases}
  ln(I_{max}) - ln(I_{raw} + 1.0) & \quad \text{if } I_{raw} \lt I_{max}\\
  0 & \quad \text{if } I_{raw} = I_{max}\\
\end{cases}\\
$

### Calculate murhot0 :
$
\mu\rho t_{ref} =
\begin{cases}
  \mu\rho t \cdot \left(\frac{t_{max}}{t_{model}}\right) & \quad \text{if } t_{model} \gt t_{min}\\
  \mu\rho t & \quad \text{if } t_{model} \le t_{min} \text{ or outside core area}\\
\end{cases}\\
\text{ }\\
t_{max} = \text{maximum ray path length in core}\\
t_{min} = \text{minimum value used for correction, to avoid numerical instability near edges}\\
$

## Contrast Enhancement

### Requirements:

Must be a function that is continuous over the input range and through the peak! Using two different functions around the peak leads to strong anomalies and unstable behavior at the peak itself, which creates artifacts over large swaths of the images.

### Histogram Stretching and Centering:
$
\text{Initial normalization}\\
\text{ }\\
X = 
\begin{cases}
  \frac{\mu\rho t_{ref} - O_{min}}{O_{max} - O_{min}} & \quad \text{if } O_{min} \le \mu\rho t_{ref} \le O_{max}\\
  0.0 & \quad \text{if } \mu\rho t_{ref} \lt O_{min}\\
  1.0 & \quad \text{if } \mu\rho t_{ref} \gt O_{max}\\
\end{cases}\\
\text{ }\\
\text{Using a polynomial to stretch the X values and center the peak at 0.5}\\
\text{ }\\
X_{peak} = \frac{O_{peak} - O_{min}}{O_{max} - O_{min}}\\
n = \frac{ln\left(0.5\right)}{ln\left(X_{peak}\right)}\\
P = X^n\\
\text{ }\\
\text{Using a sine function to further stretch the values}\\
\text{ }\\
S(X) = 0.5 \cdot \sin\left(\pi \cdot (X - 0.5)\right) + 0.5\\
S(P) = 0.5 \cdot \sin\left(\pi \cdot (P - 0.5)\right) + 0.5\\
\text{ }\\
\text{When } X_{peak} \text{ is far from 0.5, S(X) is preferred. When } X_{peak} \text{ is close to 0.5, S(P) is preferred.}\\
\text{Using S(X) will not center the peak at 0.5, but it will prevent substantial histogram distortion.}\\
\text{ }\\
w = 1.0 - \left| 2.0 \cdot \left(X_{peak} - 0.5\right)\right|^2\\
Y = w \cdot S(P) + (1-w) \cdot S(X)\\
\text{ }\\
\text{Converting to pixel values}\\
\text{ }\\
N_{stretch} = \left(\frac{N_{max} - N_{min}}{1.0}\right) \cdot Y + N_{min}\\
$

### Reverse Intensities to Match Original Greyscale Impression:
$
I_{proc} = -1*N_{stretch} + N_{max}\\
$

### Values:
$
O_{min} = ln(I_{max}) - ln(I_{high} + 1.0)\\
O_{peak} = ln(I_{max}) - ln(I_{peak} + 1.0)\\
O_{max} = ln(I_{max}) - ln(I_{low} + 1.0)\\
N_{min} = 0.0\\
N_{max} = I_{max}\\
$